In [1]:
""" Upload WHYMAP to bigquery.
-------------------------------------------------------------------------------

Author: Rutger Hofste
Date: 20181114
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

TESTING = 0
OVERWRITE_OUTPUT = 1
SCRIPT_NAME = 'Y2018M11D14_RH_WHYMAP_RDS_to_BQ_V01'
OUTPUT_VERSION = 1

BQ_PROJECT_ID = "aqueduct30"
BQ_OUTPUT_DATASET_NAME = "geospatial_v01"

RDS_DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
RDS_DATABASE_NAME = "database01"
RDS_INPUT_TABLE_NAME = "y2018m11d14_rh_whymap_to_rds_v01_v01"
BQ_OUTPUT_TABLE_NAME = "{}_v{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION).lower()

print("\nRDS_DATABASE_ENDPOINT: ", RDS_DATABASE_ENDPOINT,
      "\nRDS_DATABASE_NAME: ", RDS_DATABASE_NAME,
      "\nRDS_INPUT_TABLE_NAME: ",RDS_INPUT_TABLE_NAME,
      "\nBQ_OUTPUT_DATASET_NAME: ", BQ_OUTPUT_DATASET_NAME,
      "\nBQ_OUTPUT_TABLE_NAME: ", BQ_OUTPUT_TABLE_NAME)



RDS_DATABASE_ENDPOINT:  aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com 
RDS_DATABASE_NAME:  database01 
RDS_INPUT_TABLE_NAME:  y2018m11d14_rh_whymap_to_rds_v01_v01 
BQ_OUTPUT_DATASET_NAME:  geospatial_v01 
BQ_OUTPUT_TABLE_NAME:  y2018m11d14_rh_whymap_rds_to_bq_v01_v01


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M11D14 UTC 14:06


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import os
import sqlalchemy
import pandas as pd
import geopandas as gpd
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "aqueduct30"
client = bigquery.Client(project=BQ_PROJECT_ID)

In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = sqlalchemy.create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,RDS_DATABASE_ENDPOINT,RDS_DATABASE_NAME))


In [5]:
sql = """
SELECT
  aqid,
  geom,
  ST_AsText(geom) AS wkt
FROM
  {}
""".format(RDS_INPUT_TABLE_NAME)

In [6]:
gdf = gpd.read_postgis(sql=sql,
                       con=engine)

In [7]:
gdf.shape

(3402, 3)

In [8]:
gdf.head()

,aqid,geom,wkt
0,0,(POLYGON ((-27.61888898379225 81.4749958595083...,MULTIPOLYGON(((-27.6188889837922 81.4749958595...
1,19,(POLYGON ((-16.91555506127432 81.8560967874564...,MULTIPOLYGON(((-16.9155550612743 81.8560967874...
2,20,(POLYGON ((-27.61888898379225 81.4749958595083...,MULTIPOLYGON(((-27.6188889837922 81.4749958595...
3,25,(POLYGON ((63.47083247656703 81.58249504685597...,MULTIPOLYGON(((63.470832476567 81.582495046856...
4,34,(POLYGON ((-94.67110548100065 80.0569264559232...,MULTIPOLYGON(((-94.6711054810007 80.0569264559...


In [9]:
destination_table = "{}.{}".format(BQ_OUTPUT_DATASET_NAME,BQ_OUTPUT_TABLE_NAME)

In [10]:
destination_table

'geospatial_v01.y2018m11d14_rh_whymap_rds_to_bq_v01_v01'

In [11]:
df = pd.DataFrame(gdf.drop("geom",1))

In [12]:
df.to_gbq(destination_table=destination_table,
          project_id=BQ_PROJECT_ID,
          chunksize=100,
          if_exists="replace")

35it [02:59,  5.13s/it]


In [13]:
engine.dispose()

In [14]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:03:10.159636


previous runs:  
0:03:04.858003